In [39]:
import matplotlib.pyplot as plt
import pandas as pd
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, plot_confusion_matrix,
                             roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from imblearn.ensemble import EasyEnsembleClassifier, RUSBoostClassifier
from utils import *
import numpy as np
from collections import Counter



In [40]:
X_train_poly = pd.read_csv("../dataset/train_poly.csv")
X_test_poly = pd.read_csv("../dataset/test_poly.csv")
y_train_poly_poly = X_train_poly.match
y_test_poly = X_test_poly.match

X_train_poly = X_train_poly.drop("match",1)
X_test_poly = X_test_poly.drop("match",1)

In [41]:
parameters = {
    "learning_rate": [0.01, 0.05, 0.1],
    "min_samples_split": [0.2, 0.3],
    "min_samples_leaf": [0.3, 0.4],
    "max_depth":[3, 5, 8],
    "max_features":["log2","sqrt"],
    "subsample":[0.5, 0.8, 1.0]
    }

gs_gbc = GridSearchCV(GradientBoostingClassifier(), parameters,scoring='balanced_accuracy',refit=True,cv=2, n_jobs=-1)

In [45]:
gs_gbc.fit(X_train_poly, y_train_poly)
best_parameters = gs_gbc.best_params_

In [47]:
eec = EasyEnsembleClassifier(random_state=42,
                              n_jobs=-1,
                              base_estimator=GradientBoostingClassifier(
            max_depth=8,
            max_features='sqrt',
            min_samples_leaf=0.3,
            min_samples_split=0.3,
            n_estimators=100
                ))

eec.fit(X_train_poly, y_train_poly)

EasyEnsembleClassifier(base_estimator=GradientBoostingClassifier(max_depth=8,
                                                                 max_features='sqrt',
                                                                 min_samples_leaf=0.3,
                                                                 min_samples_split=0.3),
                       n_jobs=-1, random_state=42)

In [51]:
Y_hat_test_poly = eec.predict(X_test_poly)
y_hat_train_poly = eec.predict(X_train_poly)

print(classification_report(y_test_poly, Y_hat_test_poly))
print(classification_report(y_train_poly, y_hat_train_poly))
print(balanced_accuracy_score(y_test_poly, Y_hat_test))

# disp = plot_confusion_matrix(eec, X_test_poly, y_test_poly,
#                              cmap=plt.cm.Blues)
# disp.ax_.set_title('title')

# plt.show()

              precision    recall  f1-score   support

           0       1.00      0.68      0.81      1979
           1       0.39      1.00      0.56       402

    accuracy                           0.73      2381
   macro avg       0.69      0.84      0.68      2381
weighted avg       0.90      0.73      0.77      2381

              precision    recall  f1-score   support

           0       1.00      0.70      0.82      4617
           1       0.40      1.00      0.57       936

    accuracy                           0.75      5553
   macro avg       0.70      0.85      0.70      5553
weighted avg       0.90      0.75      0.78      5553

0.8383411391752706
